In [10]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, inspect
import os

In [11]:
engine = create_engine('sqlite:///inventory.db')

In [ ]:
for file in os.listdir('data'):
    if '.csv' in file:
        df= pd.read_csv('data/' + file)
        print(df.shape)
        ingest_db(df, file[:,-4], engine)

(206529, 9)
(224489, 9)
(2372474, 16)
(12261, 9)
(12825363, 14)
(5543, 10)


In [8]:
import pandas as pd
from sqlalchemy import create_engine
import os
import time

def ingest_data_in_chunks(file_path, table_name, engine, chunk_size=100000):
    """
    Reads a large CSV file in chunks and ingests it into a SQL database.
    
    Args:
        file_path (str): The full path to the CSV file.
        table_name (str): The name of the SQL table to create.
        engine: The SQLAlchemy engine connection.
        chunk_size (int): The number of rows per chunk.
    """
    print(f"Processing {os.path.basename(file_path)}...")
    start_time = time.time()
    
    # Create an iterator object that reads the CSV in chunks
    df_iterator = pd.read_csv(file_path, chunksize=chunk_size, low_memory=False)
    
    # Get the first chunk to create the table structure
    first_chunk = next(df_iterator)
    
    # Write the first chunk to the database, replacing any existing table
    first_chunk.to_sql(table_name, con=engine, if_exists='replace', index=False)
    
    # Now, loop through the remaining chunks and append them
    for i, chunk in enumerate(df_iterator, 1):
        chunk.to_sql(table_name, con=engine, if_exists='append', index=False)
        print(f"  - Ingested chunk {i+1} for {table_name}")
        
    end_time = time.time()
    print(f"✅ Finished ingesting '{table_name}' in {end_time - start_time:.2f} seconds.\n")


# --- Main Script ---
# Create a connection engine to the SQLite database
engine = create_engine('sqlite:///inventory.db')
data_directory = 'data'

for file_name in os.listdir(data_directory):
    if file_name.endswith('.csv'):
        # Construct the full file path
        full_path = os.path.join(data_directory, file_name)
        
        # Create a clean table name by removing the '.csv' extension
        table_name = file_name[:-4] 
        
        # Call the ingestion function
        ingest_data_in_chunks(file_path=full_path, table_name=table_name, engine=engine)

print("All files have been successfully ingested into the database.")

Processing begin_inventory.csv...
  - Ingested chunk 2 for begin_inventory
  - Ingested chunk 3 for begin_inventory
✅ Finished ingesting 'begin_inventory' in 2.95 seconds.

Processing end_inventory.csv...
  - Ingested chunk 2 for end_inventory
  - Ingested chunk 3 for end_inventory
✅ Finished ingesting 'end_inventory' in 2.91 seconds.

Processing purchases.csv...
  - Ingested chunk 2 for purchases
  - Ingested chunk 3 for purchases
  - Ingested chunk 4 for purchases
  - Ingested chunk 5 for purchases
  - Ingested chunk 6 for purchases
  - Ingested chunk 7 for purchases
  - Ingested chunk 8 for purchases
  - Ingested chunk 9 for purchases
  - Ingested chunk 10 for purchases
  - Ingested chunk 11 for purchases
  - Ingested chunk 12 for purchases
  - Ingested chunk 13 for purchases
  - Ingested chunk 14 for purchases
  - Ingested chunk 15 for purchases
  - Ingested chunk 16 for purchases
  - Ingested chunk 17 for purchases
  - Ingested chunk 18 for purchases
  - Ingested chunk 19 for purc

In [12]:

# --- Configuration ---
db_path = 'sqlite:///inventory.db'

# --- Script ---
engine = create_engine(db_path)
inspector = inspect(engine)

# Get the list of all table names in the database
table_names = inspector.get_table_names()

print(f"Inspecting database: {db_path.split('/')[-1]}\n" + "="*40)

# Loop through each table to get its dimensions
for table in table_names:
    # Query to count rows (very memory-efficient)
    row_count_query = f"SELECT COUNT(*) FROM {table}"
    row_count = pd.read_sql_query(row_count_query, engine).iloc[0, 0]
    
    # Query to count columns using SQLite's PRAGMA (also memory-efficient)
    col_count_query = f"PRAGMA table_info({table});"
    col_count = len(pd.read_sql_query(col_count_query, engine))
    
    # Print the results in a formatted way
    print(f"Table: {table:<20} | Rows: {row_count:<10} | Columns: {col_count}")

print("="*40)

Inspecting database: inventory.db
Table: begin_inventory      | Rows: 206529     | Columns: 9
Table: end_inventory        | Rows: 224489     | Columns: 9
Table: purchase_prices      | Rows: 12261      | Columns: 9
Table: purchases            | Rows: 2372474    | Columns: 16
Table: sales                | Rows: 12825363   | Columns: 14
Table: vendor_invoice       | Rows: 5543       | Columns: 10
